In [0]:
# %sql
# CREATE TABLE IF NOT EXISTS workspace.streaming_test.json_events_delta (
#     event_time STRING,
#     user_id STRING,
#     action STRING, 
#     value DOUBLE
# )
# USING DELTA;

In [0]:
# dbutils.fs.ls("s3://test-streaming-12354325/")

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

schema = StructType([
    StructField("event_time", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("action", StringType(), True),
    StructField("value", DoubleType(), True)  # <-- Forzamos que sea double
])

json_path = "s3a://test-streaming-12354325/streaming_jsons/"
schema_path = "dbfs:/tmp/streaming_schema/"  # Ajusta la ruta según tu caso

df_stream = (spark.readStream
                  .format("cloudFiles")                   
                  .option("cloudFiles.format", "json")     
                  .option("cloudFiles.inferSchema", "true") 
                  .option("cloudFiles.schemaLocation", schema_path)
                  .schema(schema)
                  .load(json_path))




In [0]:
# spark.table("workspace.streaming_test.json_events_delta").printSchema()

In [0]:
# df_stream.printSchema()

In [0]:
checkpoint_path = "dbfs:/mnt/path_to_checkpoint/json_events_checkpoint"

query = (
    df_stream
    .writeStream
    .format("delta")
    .option("checkpointLocation", checkpoint_path)
    .outputMode("append")
    .table("workspace.streaming_test.json_events_delta")  # <-- En vez de .start("ruta...")
)

query.awaitTermination()